<a href="https://colab.research.google.com/github/takagiyuusuke/Q_TETRIS_lv1/blob/main/Q_miniTETRIS_O%E3%83%9F%E3%83%8E%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. インポート・乱数シードの設定

In [ ]:
import random

import torch
import numpy as np


random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

# 1. miniTETRISのロジック
横6マスのテトリスを定義する

In [71]:
import time
import random
import copy

class Tetris:
  #各種ブロックの定義
  block1 = [[-1, 2, 1], [0, 2, 1], [-1, 3, 1], [0, 3, 1], [-0.5, 2.5]] #O
  block2 = [[0, 2, 2], [0, 3, 2], [0, 4, 2], [-1, 3, 2], [0, 3]] #T
  block3 = [[-1, 1, 3], [-1, 2, 3], [-1, 3, 3], [-1, 4, 3], [-0.5, 2.5]] #I
  block4 = [[0, 2, 4], [0, 3, 4], [0, 4, 4], [-1, 2, 4], [0, 3]] #J
  block5 = [[-1, 4, 5], [0, 2, 5], [0, 3, 5], [0, 4, 5], [0, 3]] #L
  block6 = [[-1, 2, 6], [-1, 3, 6], [0, 3, 6], [0, 4, 6], [0, 3]] #Z
  block7 = [[0, 2, 7], [0, 3, 7], [-1, 3, 7], [-1, 4, 7], [0, 3]] #S
  blocks =[block1,block2,block3,block4,block5,block6,block7]

  #ブロックの回転の定義
  roundblock=[[[[0,-1],[-1,0],[0,1]],[[-1,0],[0,1],[1,0]],[[0,1],[1,0],[0,-1]],[[1,0],[0,-1],[-1,0]]],
  [[[-0.5,-1.5],[-0.5,-0.5],[-0.5,0.5],[-0.5,1.5]],[[-1.5,0.5],[-0.5,0.5],[0.5,0.5],[1.5,0.5]],[[0.5,-1.5],[0.5,-0.5],[0.5,0.5],[0.5,1.5]],[[-1.5,-0.5],[-0.5,-0.5],[0.5,-0.5],[1.5,-0.5]]],
  [[[-1,-1],[0,-1],[0,1]],[[-1,1],[-1,0],[1,0]],[[0,-1],[0,1],[1,1]],[[-1,0],[1,0],[1,-1]]],
  [[[-1,1],[0,1],[0,-1]],[[1,0],[-1,0],[1,1]],[[0,-1],[0,1],[1,-1]],[[1,0],[-1,0],[-1,-1]]],
  [[[-1,-1],[-1,0],[0,1]],[[-1,1],[0,1],[1,0]],[[0,-1],[1,0],[1,1]],[[1,-1],[0,-1],[-1,0]]],
  [[[0,-1],[-1,0],[-1,1]],[[-1,0],[0,1],[1,1]],[[1,-1],[1,0],[0,1]],[[-1,-1],[0,-1],[1,0]]]]

  #ブロックを回転する際に移動する座標を定義(詳しくはSRS)
  SRS =[[[0,0],[0,1],[-1,1],[2,0],[2,1]],[[0,0],[0,-1],[-1,-1],[2,0],[2,-1]],[[0,0],[0,-1],[1,-1],[-2,0],[-2,-1]],[[0,0],[0,1],[1,1],[-2,0],[-2,1]]]
  SRSofI =[[[0,0],[0,-1],[0,2],[-2,-1],[1,2]],[[0,0],[0,-2],[0,1],[1,-2],[-2,1]],[[0,0],[0,1],[0,-2],[1,2],[-1,-2]],[[0,0],[0,2],[0,-1],[-1,2],[2,-1]]]

  '''
  盤面の情報などの情報の初期化を行う
  display:画面を表示するか否か
  '''
  def __init__(self, display: bool):
    self.display = display
    self.reset_game()

  '''
  ゲームをリセットする
  '''
  def reset_game(self):
    self.location=[]  #現在の盤面
    self.moveblock=[] #現在落下中のブロック
    self.supports=[]  #落下後の場所を示す補助ブロック
    self.hold=[]      #ホールド中のブロック
    self.angle = 0    #落下中のブロックの角度(0,1,2,3)
    self.num=1        #落下中のブロックが何番目か
    self.lines = 0    #消したラインの数
    self.score = 0    #スコア
    self.level = 0    #レベル
    self.canhold = True  #ホールドできるか
    # self.nex =[random.choice(Tetris.blocks) for i in range(20)] #後におちるブロック
    self.nex =[Tetris.block1 for i in range(20)]


    self.setblock(copy.deepcopy(self.nex[0])) #最初のブロックをセットする

  '''
  ゲームの盤面を表示する
  '''
  def printboad(self, display_force = False):
      if display_force:
        pass
      elif not self.display:
        return
      boad = "\n"
      boad += ("⏹"*8+"     NEXT "+str(self.num+1) +"\n")
      for i in range(22):
          boad += ("⏹")
          for j in range(6):
              ad = [l for l in self.location + self.moveblock if l[0] ==i and l[1]==j]
              if ad:
                  if ad[0][2] ==0:
                      boad += ("💥")
                  else:
                      boad += (chr(128996+ad[0][2]))
              else:
                  ae = [m for m in self.supports if m[0] ==i and m[1]==j]
                  if ae:
                      boad += ("⬛")
                  else:
                      boad += ("⬜")
          boad += ("⏹  ")
          if i == 0 or i == 1:
              for j in range(1,5):
                  af = [l for l in self.nex[self.num%20] if l[0] ==i-1 and l[1]==j]
                  if af:
                      boad += (chr(128996+af[0][2]))
                  else:
                      boad += ("⬜")
          elif i == 3:
              boad += (" 👆👆👆 "+str(self.num+2))
          elif i == 4 or i == 5:
              for j in range(1,5):
                  af = [l for l in self.nex[(self.num+1)%20] if l[0] ==i-5 and l[1]==j]
                  if af:
                      boad += (chr(128996+af[0][2]))
                  else:
                      boad += ("⬜")
          elif i == 7:
              boad += (" 👆👆👆 "+str(self.num+3))
          elif i == 8 or i == 9:
              for j in range(1,5):
                  af = [l for l in self.nex[(self.num+2)%20] if l[0] ==i-9 and l[1]==j]
                  if af:
                      boad += (chr(128996+af[0][2]))
                  else:
                      boad += ("⬜")
          elif i == 12:
              boad += (" LEVEL =>"+str(self.level))
          elif i ==14:
              boad += (" LINES =>"+str(self.lines))
          elif i == 16:
              boad += (" SCORE =>"+str(self.score))
          elif i == 18:
              boad += (" -HOLD-")
          elif i == 19 or i == 20:
              for j in range(1,5):
                  af = [l for l in self.hold if l[0] ==i-20 and l[1]==j]
                  if af:
                      boad += (chr(128996+af[0][2]))
                  else:
                      boad += ("⬜")
          elif i == 21:
              boad += (" ------")
          boad += ("\n")
      boad += ("⏹"*8)
      print(boad)

  '''
  引数に指定したブロックを盤面に追加する
  '''
  def setblock(self, block):
      self.moveblock.extend(block)
      self.angle = 0
      self.supportdisplay()
      if self.num%20==13:
          # self.nex =[(random.choice(Tetris.blocks) if i < 10 else self.nex[i]) for i in range(20)]
          self.nex = [(Tetris.block1 if i < 10 else self.nex[i]) for i in range(20)]
      elif self.num%20==3:
          # self.nex =[(random.choice(Tetris.blocks) if i >= 10 else self.nex[i]) for i in range(20)]
          self.nex = [(Tetris.block1 if i >= 10 else self.nex[i]) for i in range(20)]

  '''
  ブロックを1段落下させる
  '''
  def fallblock(self):
      # global moveblock
      for i in self.moveblock:
          i[0] +=1

  '''
  ブロックが落下しなくなったか判定する
  '''
  def stopfall(self):
      for i in self.moveblock:
          aa = [j for j in self.location if (j[0] ==i[0]+1 and j[1]==i[1])]
          if len(aa)>0 or i[0] >=21:
              return True
      return False

  '''
  一列そろっている箇所があれば消去する
  '''
  def removeline(self):
      # global location,lines,score,level,supports
      lines_first = self.lines
      L = []
      max_colum = 0
      for i in range(22):
          a = [False for l in range(6)]
          for j in range(6):
              ab = [k for k in self.location if k[0]==i and k[1]==j]
              if len(ab) > 0:
                  a[j] = True
          if all(a):
              self.location = [l for l in self.location if l[0] != i]
              self.location.extend([[i,k,0] for k in range(6)])
              L.append(i)
              self.lines += 1
              if self.lines % 10 == 0:
                  self.level += 1
      if L:
              # self.printboad()
              self.location = [l for l in self.location if l[0] not in L]
              self.supports = []
              for j in L:
                  for m in self.location:
                      if m[0] < j:
                          m[0] += 1
              # self.printboad()
      lines_final = self.lines
      self.score += (lines_final-lines_first)**2*1000*(self.level+1)
      # self.score += 100

  '''
  このまま落下したときに辿り着く地点に補助ブロックを表示する
  '''
  def supportdisplay(self):
      if not self.display:
        return

      z = copy.deepcopy(self.moveblock)
      while True:
          d = True
          for i in z:
              aa = [j for j in self.location if (j[0] ==i[0]+1 and j[1]==i[1])]
              if len(aa)>0 or i[0] >=21:
                  d = False
                  break
          if not d:
              self.supports = [[z[0][0],z[0][1],8],[z[1][0],z[1][1],8],[z[2][0],z[2][1],8],[z[3][0],z[3][1],8]]
              break
          else:
              for i in z:
                  i[0] +=1

  '''
  ブロックを右に動かせるか確かめたのちに右に移動させる
  '''
  def right(self):
      # global moveblock
      ok = True
      for i in self.moveblock:
          aa = [j for j in self.location if (j[1] ==i[1]+1 and j[0]==i[0])]
          if len(aa) > 0 or i[1] >= 5:
              ok = False
      if ok:
          for j in self.moveblock:
              j[1] +=1
          self.supportdisplay()
          self.printboad()

  '''
  ブロックを左に動かせるか確かめたのちに左に移動させる
  '''
  def left(self):
      # global moveblock
      ok = True
      for i in self.moveblock:
          aa = [j for j in self.location if (j[1] ==i[1]-1 and j[0]==i[0])]
          if len(aa) > 0 or i[1] <=0:
              ok = False
      if ok:
          for j in self.moveblock:
              j[1] -=1
          self.supportdisplay()
          self.printboad()

  '''
  ブロックを下に動かせるか確かめたのちに下に移動させる
  '''
  def down(self):
      if self.stopfall():
            if self.gameover():
                # self.printboad
                # self.location.append(self.moveblock)
                # self.moveblock = []
                # self.gameover_performance() if self.display else None
                return
            self.moveblock = [x for x in self.moveblock if len(x) != 2]
            self.location.extend(self.moveblock)
            self.location
            self.moveblock = []
            self.removeline()
            self.setblock(copy.deepcopy(self.nex[self.num%20]))
            self.canhold = True
            self.num += 1
      else:
          for j in self.moveblock:
              j[0] +=1
          self.printboad()

  '''
  ブロックを回転させる。無理だったらそのままの状態にする
  - direction:右か左か
  '''
  def turn(self, direction):
      # global moveblock,angle
      col = self.moveblock[0][2]
      if col > 1:
          angle_before = self.angle
          if direction == "left":
              self.angle = self.angle-1 if self.angle >0 else 3
          elif direction == "right":
              self.angle = self.angle+1 if self.angle <3 else 0
          e = Tetris.roundblock[col-2][self.angle]
          if col != 3:
              SRSn = (3 if angle_before == 1 else 2)if angle_before%2==1 else(1 if self.angle == 1 else 0)
              n = 0
              while True:
                  m = [[self.moveblock[4][0]+e[0][0]+Tetris.SRS[SRSn][n][0],self.moveblock[4][1]+e[0][1]+Tetris.SRS[SRSn][n][1]],
                                  [self.moveblock[4][0]+e[1][0]+Tetris.SRS[SRSn][n][0],self.moveblock[4][1]+e[1][1]+Tetris.SRS[SRSn][n][1]],
                                  [self.moveblock[4][0]+e[2][0]+Tetris.SRS[SRSn][n][0],self.moveblock[4][1]+e[2][1]+Tetris.SRS[SRSn][n][1]],
                                  [self.moveblock[4][0]+Tetris.SRS[SRSn][n][0],self.moveblock[4][1]+Tetris.SRS[SRSn][n][1]]]
                  ac = [j for j in self.location if (j[0] ==m[0][0] and j[1] ==m[0][1])or(j[0] ==m[1][0] and j[1] ==m[1][1])or
                      (j[0] ==m[2][0] and j[1] ==m[2][1])or(j[0] ==m[3][0] and j[1] ==m[3][1])]

                  if len(ac) ==0 and max(m[0][1],m[1][1],m[2][1],m[3][1]) <= 5 and min(m[0][1],m[1][1],m[2][1],m[3][1])>=0 and max(m[0][0],m[1][0],m[2][0],m[3][0])<=21:
                      self.moveblock = [[m[0][0],m[0][1],col],[m[1][0],m[1][1],col],[m[2][0],m[2][1],col],[m[3][0],m[3][1],col],m[3]]
                      self.supportdisplay()
                      self.printboad()
                      break
                  elif n==4:
                      break
                  else:
                      n += 1
          elif col ==3:
              if angle_before == 0:
                  if self.angle == 1:
                      SRSn = 1
                  else:
                      SRSn = 0
              elif angle_before == 1:
                  if self.angle ==0:
                      SRSn = 3
                  else:
                      SRSn = 0
              elif angle_before == 2:
                  if self.angle == 1:
                      SRSn = 2
                  else:
                      SRSn = 3
              else:
                  if self.angle ==0:
                      SRSn = 2
                  else:
                      SRSn = 1
              n = 0
              while True:
                  m = [[round(self.moveblock[4][0]+e[0][0]+Tetris.SRSofI[SRSn][n][0]),round(self.moveblock[4][1]+e[0][1]+Tetris.SRSofI[SRSn][n][1])],
                                  [round(self.moveblock[4][0]+e[1][0]+Tetris.SRSofI[SRSn][n][0]),round(self.moveblock[4][1]+e[1][1]+Tetris.SRSofI[SRSn][n][1])],
                                  [round(self.moveblock[4][0]+e[2][0]+Tetris.SRSofI[SRSn][n][0]),round(self.moveblock[4][1]+e[2][1]+Tetris.SRSofI[SRSn][n][1])],
                                  [round(self.moveblock[4][0]+e[3][0]+Tetris.SRSofI[SRSn][n][0]),round(self.moveblock[4][1]+e[3][1]+Tetris.SRSofI[SRSn][n][1])]]
                  ac = [j for j in self.location if (j[0] ==m[0][0] and j[1] ==m[0][1])or(j[0] ==m[1][0] and j[1] ==m[1][1])or
                      (j[0] ==m[2][0] and j[1] ==m[2][1])or(j[0] ==m[3][0] and j[1] ==m[3][1])]

                  if len(ac) ==0 and max(m[0][1],m[1][1],m[2][1],m[3][1])<=5 and min(m[0][1],m[1][1],m[2][1],m[3][1])>=0 and max(m[0][0],m[1][0],m[2][0],m[3][0])<=21:
                      self.moveblock = [[m[0][0],m[0][1],col],[m[1][0],m[1][1],col],[m[2][0],m[2][1],col],[m[3][0],m[3][1],col],[self.moveblock[4][0]+Tetris.SRSofI[SRSn][n][0],self.moveblock[4][1]+Tetris.SRSofI[SRSn][n][1]]]
                      self.supportdisplay()
                      self.printboad()
                      break
                  elif n==4:
                      break
                  else:
                      n += 1

  '''
  ブロックをホールドする
  '''
  def holding(self):
      # global hold,moveblock,canhold,num
      if not self.hold:
          self.hold =Tetris.blocks[self.moveblock[0][2]-1][:]
          self.moveblock = []
          self.setblock(copy.deepcopy(self.nex[self.num%20]))
          self.canhold = True
          self.num += 1
      else:
          if self.canhold:
              #print(Tetris.blocks,moveblock)
              self.hold , self.moveblock = copy.deepcopy(Tetris.blocks[self.moveblock[0][2]-1]) , copy.deepcopy(self.hold)
              self.canhold = False
              self.supportdisplay()
          else:
              return
      self.printboad()

  '''
  ゲームオーバーかどうかの判定を行う
  '''
  def gameover(self):
    try:
      min_moveblock = min(self.moveblock[i][0] for i in range(len(self.moveblock)))
    except:
      min_moveblock = 22
    try:
      min_location = min(self.location[i][0] for i in range(len(self.location)))
    except:
      min_location = 22

    return min_moveblock < 2 and min_location < 3

  '''
  ゲームオーバー時の演出
  '''
  def gameover_performance(self):
      # global location,supports
      for i in range(22):
          self.supports.extend([x for x in self.location if x[0] == i])
          self.location = [x for x in self.location if x[0] != i]
          self.printboad() if self.display else None
          time.sleep(0.1) if self.display else None
      self.location = [[x[0], x[1], 0 ]for x in self.supports]
      self.printboad()
      time.sleep(0.1) if self.display else None
      self.location = []
      self.supports = []
      self.printboad()
      return

  '''
  盤面と落下中のブロック、ホールドブロックの形状を返す
  '''
  def get_state(self):
    # 形状のみを抽出して返す関数。
    def transform_matrix(matrix):
      result = [0 for i in range (6)]
      if len(matrix) == 0:
        return result
      maxe0 = min(matrix[i][0] for i in range(len(matrix)-1))
      for e in matrix:
          if e[0] == round(maxe0):
              result[round(e[1])] = 1
      return result
    location = tuple(transform_matrix(self.location))
    # moveblock = torch.Tensor(transform_matrix(self.moveblock, False))
    moveblock = self.moveblock[0][2], self.angle, int(self.moveblock[4][1])
    # hold = torch.Tensor(transform_matrix(self.hold, True))
    return location, moveblock

  '''
  すべての可能な行動を返す
  '''
  def get_possible_actions(self):
    return ["down", "left", "right", "turn-left", "turn-right", "hold"]

  '''
  スコアを取得する
  '''
  def get_score(self):
    return self.score

  '''
  盤面の変更を行う
  '''
  def modify_board(self, action):
    # self.score -= 1
    if action == "right":
        self.right()
    elif action == "left":
        self.left()
    elif action == "down":
        self.down()
    elif action == "turn-left":
        self.turn("left")
    elif action == "turn-right":
        self.turn("right")
    elif action == "hold":
        self.holding()


# 2. Agentの定義

In [67]:
class Agent:
    def __init__(self):
        self.frozen = False

    def train(self):
        self.frozen = False

    def eval(self):
        self.frozen = True

    def _observe(self, tetris: Tetris ) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        return tetris.get_state()

    def action(self, tetris: Tetris):
        pass

    def update(self, tetris: Tetris, state, action, reward1, new_state):
        pass

## 2-1. Qエージェントの定義

In [68]:
class QAgent(Agent):
    def __init__(self, lr: float, eps = 0.3):
        super().__init__()
        self.q_table = {}  # Qテーブルの初期化
        self.discount_factor = 0.9
        self.learning_rate = lr
        self.epsilon = eps

    def action(self, tetris: Tetris) :
        board, block = self._observe(tetris)
        possible_actions = tetris.get_possible_actions()
        if not self.frozen and random.random() < self.epsilon:
            return random.choice(possible_actions)
        else:
            best_action, _ = self._get_the_best(board, block, possible_actions)
            return best_action

    def _get_the_best(self, board, block, possible_moves):
        best_move = None
        best_q_value = -float('inf')
        for move in possible_moves:
            q_value = self.q_table.get((board,block,move), 0)
            if q_value > best_q_value:
                best_q_value = q_value
                best_move = move
        return best_move, best_q_value

    #stateとnext_stateはboardとblockを渡す
    def update(self, tetris, state, action, reward, next_state):
        if self.frozen:
            return None

        old_value = self.q_table.get((state[0], state[1], action), 0)
        possible_moves = tetris.get_possible_actions()

        assert next_state is not None
        next_max = max([self.q_table.get((next_state[0], next_state[1], next_action), 0) for next_action in possible_moves])

        new_value = old_value + self.learning_rate * (reward + self.discount_factor * next_max - old_value)
        self.q_table[(state[0], state[1], action)] = new_value

## 2-2. 人間エージェントの定義

In [ ]:
class HumanAgent(Agent):
  def action(self, tetris: Tetris):
        valid_moves = tetris.get_possible_actions()
        while True:
            user_input = input("Enter your move: ")
            return user_input

# 3. Env.

In [69]:
import tqdm
import plotly.graph_objects as go

class Env:
    def __init__(self, agent: Agent, tetris: Tetris) -> None:
        self.agent = agent
        self.tetris = tetris

    def _get_reward(self) -> int:
        score = self.tetris.get_score()
        return score

    def train(self, episodes: int, visualize=False) :  # Nエピソード実行
        record = []
        for i in tqdm.tqdm(range(episodes)):
            score = self.execute(train=True, visualize=visualize)
            record.append(self._get_reward())
        print("Report:")
        return record

    def execute(self, train=False, visualize=True):  # 1 episode
        self.tetris.reset_game()

        while not self.tetris.gameover():
            state = self.tetris.get_state()
            while True:
                action = self.agent.action(self.tetris)
                before_point = self.tetris.get_score()
                self.tetris.modify_board(action)
                after_point = self.tetris.get_score()
                reward = after_point - before_point
                if train:
                    next_state = self.tetris.get_state()

                    if action is not None:
                        self.agent.update(self.tetris, state, action, reward, next_state)
                break

        # game end
        score = self.tetris.get_score()
        if random.random() < 0.01:
          self.tetris.printboad(display_force = True)
        return score

# 4. Q学習
学習中、現状確認のために1%の確率でゲームオーバー時の盤面が出力される

In [ ]:
q_agent = QAgent(lr=0.1, eps=0.1)
q_agent.epsilon = 0.05
tetris = Tetris(False)
env = Env(q_agent, tetris)
record = env.train(1000)
fig = go.Figure(data=go.Scatter(y=record))
fig.show()

# 5. 学習が完了したQエージェントにプレイさせてみる
モデルが実際にどのようなプレイをするのか確かめる。

In [ ]:
q_agent.eval()
env = Env(q_agent, Tetris(True))
env.execute()
q_agent.train()

# 6. 追加で学習させたい場合
モデルをリセットせずにさらに学習させたい場合はここを実行  
epsilon, lr を適宜変更するとよい

In [ ]:
tetris = Tetris(False)
q_agent.epsilon = 0.05
q_agent.learning_rate = 0.1
env = Env(q_agent, tetris)
record = env.train(1000)
fig = go.Figure(data=go.Scatter(y=record))
fig.show()

# 付録

## 手動で遊んでみる
※ 非常にユーザビリティが悪いため、やらない方がよい(笑)


In [ ]:
env = Env(HumanAgent(), Tetris(True))
env.execute()

## Qテーブルを見てみる
モデルが学習したQ値を見ることができる。ただし、モデルが重い場合は非常に時間がかかるため実行しない方がよい。


In [ ]:
print((q_agent.q_table))